In [ ]:
# Arc2Face 클론 & 디렉토리 이동
!git clone https://github.com/foivospar/Arc2Face.git
%cd Arc2Face

In [ ]:
# 1) numpy (1.23.x)
%pip install "numpy<1.24.0"

# 2) torch + torchvision (GPU 빌드)
%pip install torch==2.0.1 torchvision==0.15.2 \
    -f https://download.pytorch.org/whl/cu118/torch_stable.html

# 3) Arc2Face 요구사항
%pip install diffusers==0.23.0 \
            transformers==4.34.1 \
            peft \
            accelerate \
            insightface \
            onnxruntime-gpu \
            gradio

In [ ]:
from huggingface_hub import hf_hub_download

hf_hub_download(repo_id="FoivosPar/Arc2Face", filename="arc2face/config.json", local_dir="./models")
hf_hub_download(repo_id="FoivosPar/Arc2Face", filename="arc2face/diffusion_pytorch_model.safetensors", local_dir="./models")
hf_hub_download(repo_id="FoivosPar/Arc2Face", filename="encoder/config.json", local_dir="./models")
hf_hub_download(repo_id="FoivosPar/Arc2Face", filename="encoder/pytorch_model.bin", local_dir="./models")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Colab에서 실행
!mkdir -p models/antelopev2
!unzip "/content/drive/MyDrive/antelopev2.zip" -d models/antelopev2

In [ ]:
hf_hub_download(repo_id="FoivosPar/Arc2Face", filename="arcface.onnx", local_dir="./models/antelopev2")

In [ ]:
%pip uninstall -y peft
%pip install peft==0.5.0

In [ ]:
# 1) Huggingface Hub 버전 고정 (cached_download + split_* 모두 포함)
%pip install huggingface_hub==0.17.3

# 2) Accelerate 버전도 호환되는 버전으로 고정
%pip install accelerate==0.20.3


In [ ]:
import sys, os
# Arc2Face 클론 경로에 맞게 변경
sys.path.append(os.path.abspath("/content/Arc2Face"))

from diffusers import (
    StableDiffusionPipeline,
    UNet2DConditionModel,
    DPMSolverMultistepScheduler,
)

from arc2face import CLIPTextModelWrapper, project_face_embs

import torch
from insightface.app import FaceAnalysis
from PIL import Image
import numpy as np

# Arc2Face is built upon SD1.5
# The repo below can be used instead of the now deprecated 'runwayml/stable-diffusion-v1-5'
base_model = 'stable-diffusion-v1-5/stable-diffusion-v1-5'

encoder = CLIPTextModelWrapper.from_pretrained(
    'models', subfolder="encoder", torch_dtype=torch.float16
)

unet = UNet2DConditionModel.from_pretrained(
    'models', subfolder="arc2face", torch_dtype=torch.float16
)

pipeline = StableDiffusionPipeline.from_pretrained(
        base_model,
        text_encoder=encoder,
        unet=unet,
        torch_dtype=torch.float16,
        safety_checker=None
    )

In [ ]:
pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)
pipeline = pipeline.to('cuda')

In [ ]:
from PIL import Image
from IPython.display import display
# 원본 이미지 표시
orig = Image.open('/content/Arc2Face/assets/examples/joacquin.png')

# 최대 가로·세로를 256px로 줄이되 비율 유지
orig_small = orig.copy()
orig_small.thumbnail((256, 256))

# 축소된 이미지 표시
display(orig_small)

app = FaceAnalysis(providers=["CUDAExecutionProvider","CPUExecutionProvider"])
app.prepare(ctx_id=0, det_size=(640,640))

img = np.array(Image.open('/content/Arc2Face/assets/examples/joacquin.png'))[:,:,::-1]

faces = app.get(img)
faces = sorted(faces, key=lambda x:(x['bbox'][2]-x['bbox'][0])*(x['bbox'][3]-x['bbox'][1]))[-1]  # select largest face (if more than one detected)
id_emb = torch.tensor(faces['embedding'], dtype=torch.float16)[None].cuda()
id_emb = id_emb/torch.norm(id_emb, dim=1, keepdim=True)   # normalize embedding
id_emb = project_face_embs(pipeline, id_emb)    # pass through the encoder

In [ ]:
num_images = 4
images = pipeline(prompt_embeds=id_emb, num_inference_steps=25, guidance_scale=3.0, num_images_per_prompt=num_images).images

In [ ]:
# 셸에서 실행 → PEFT 제거
%pip uninstall -y peft


In [ ]:
# zero_emb 생성 (1×512 크기의 제로 벡터)
zero_emb = torch.zeros((1, 512), dtype=torch.float16, device="cuda")
# project_face_embs 로 negative prompt embedding 생성
neg_emb  = project_face_embs(pipeline, zero_emb)


In [ ]:
# 1) Img2Img 파이프라인 준비
from diffusers import StableDiffusionImg2ImgPipeline
from torch import Generator
from IPython.display import display

img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
    base_model,
    text_encoder=encoder,
    unet=unet,
    scheduler=pipeline.scheduler,
    torch_dtype=torch.float16
).to("cuda")

# 2) 호출 시 image=init 으로 넘겨야 init 이미지가 제대로 들어감
from torch import Generator
gen = Generator("cuda").manual_seed(1234)
init = orig.resize((512,512))

outputs = img2img(
    image                   = init,
    prompt_embeds           = id_emb,
    negative_prompt_embeds  = neg_emb,
    strength                = 0.3,
    num_inference_steps     = 50,
    guidance_scale          = 7.5,
    num_images_per_prompt   = 4,
    generator               = gen
)

# 3) 결과 표시
for img in outputs.images:
    img.thumbnail((256,256))
    display(img)
